In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sklearn
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
!pip install konlpy 
from konlpy.tag import Okt

!pip install git+https://github.com/ssut/py-hanspell.git
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
from hanspell import spell_checker
from pykospacing import spacing

!pip install konlpy 
from konlpy.tag import Okt
okt = Okt()

import regex as re

     |████████████████████████████████| 19.4MB 49.1MB/s 
     |████████████████████████████████| 460kB 30.0MB/s 
     |████████████████████████████████| 92kB 8.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-cbbng2we
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-cbbng2we
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp37-none-any.whl size=4854 sha256=eee1a621dbdcf0b9d8276e4e5a38726fbf78d00d332efc354e41c02e9916554c
  Stored in directory: /tmp/pip-ephem-wheel-cache-q49uvnuc/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-qnygxc6g
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-qnygxc6g
   

In [3]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
# NLTK Data 다운
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = open('/content/drive/MyDrive/신입프로젝트/korean_stopwords.txt').read()


# 위의 불용어는 명사가 아닌 단어 중에서 저자가 임의로 선정한 것으로 실제 의미있는 선정 기준이 아님
stop_words=stop_words.split('\n')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
txt = '커피가 너무 맛있어요!! 최고의 디저트,&케이크 진짜 달고6 맛있어요. 알바생이 친절합니다!'

In [5]:
def clean_text(texts): 
  corpus = [] 
  for i in range(0, len(texts)): 
    review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation 
    review = re.sub(r'\d+','', str(texts[i]))# remove number 
    review = review.lower() #lower case 
    review = re.sub(r'\s+', ' ', review) #remove extra space 
    review = re.sub(r'<[^>]+>','',review) #remove Html tags 
    review = re.sub(r'\s+', ' ', review) #remove spaces 
    review = re.sub(r"^\s+", '', review) #remove space from start 
    review = re.sub(r'\s+$', '', review) #remove space from the end 
    review = re.sub(r'&', '', review)

    corpus.append(review) 
  return corpus


In [6]:
clean_text(txt)

['커',
 '피',
 '가',
 '',
 '너',
 '무',
 '',
 '맛',
 '있',
 '어',
 '요',
 '!',
 '!',
 '',
 '최',
 '고',
 '의',
 '',
 '디',
 '저',
 '트',
 ',',
 '',
 '케',
 '이',
 '크',
 '',
 '진',
 '짜',
 '',
 '달',
 '고',
 '',
 '',
 '맛',
 '있',
 '어',
 '요',
 '.',
 '',
 '알',
 '바',
 '생',
 '이',
 '',
 '친',
 '절',
 '합',
 '니',
 '다',
 '!']

In [7]:
def grammar_check(text):
  spelled_sent = spell_checker.check(text)
  hanspell_sent = spelled_sent.checked
  return hanspell_sent

In [8]:
def tokenize_tagged(text):
  temp_X = okt.pos(text, norm=True, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stop_words] # 불용어 제거
  return ['/'.join(t) for t in temp_X]

In [9]:
def preprocess(crude_text):
  token_list = []
  txt = ''.join(clean_text(crude_text))
  txt1 = spacing(txt)
  txt2 = grammar_check(txt1)
  txt3 = tokenize_tagged(txt2)
  regex1 = re.compile('Josa$')
  regex2 = re.compile('Punctuation$')
  regex3 = re.compile('Suffix$')
  regex4 = re.compile('KoreanParticle$')
  regex5 = re.compile('Alpha$')
  regex6 = re.compile('Foreign$')
  text_nj = []
  for item in txt3:
    mo1 = regex1.search(item)
    mo2 = regex2.search(item)
    mo3 = regex3.search(item)
    mo4 = regex4.search(item)
    mo5 = regex5.search(item)
    mo6 = regex6.search(item)
    if mo1 == None and mo2 == None and mo3 == None and mo4 == None and  mo5 == None and mo6 == None:
      text_nj.append(item)

  return text_nj

In [10]:
preprocess(txt)

['커피/Noun',
 '너무/Adverb',
 '맛있다/Adjective',
 '최고/Noun',
 '디저트/Noun',
 '케이크/Noun',
 '진짜/Noun',
 '달/Noun',
 '맛있다/Adjective',
 '아르바이트생/Noun',
 '친절하다/Adjective']